In [72]:
import pandas as pd #importing essential library 
import numpy as np 

In [10]:
#renamed the files to make it more accessible and intuitive.
#reading csv files into pd dataframes for manipulation and data wrangling
cardholder_acc = pd.read_csv('../csvs_datathon/accy_dim.csv')
statement_data = pd.read_csv('../csvs_datathon/statement_fact.csv')
transaction_data = pd.read_csv('../csvs_datathon/transaction_fact.csv')
wrld_transaction_data = pd.read_csv('../csvs_datathon/wrld_stor_tran_fact.csv')
customer_id = pd.read_csv('../csvs_datathon/syf_id.csv')
acc_lvl_features = pd.read_csv('../csvs_datathon/rams_batch_cur.csv')
fraud_claim_case = pd.read_csv('../csvs_datathon/fraud_claim_case.csv')
fraud_claim_tran = pd.read_csv('../csvs_datathon/fraud_claim_tran.csv')

In [175]:
#goal to the first part is to predict transaction amt 

transaction_data_sales = transaction_data[(transaction_data['transaction_type'] != 'PAYMENT')]
wrld_transaction_data_sales = wrld_transaction_data[(wrld_transaction_data['transaction_type'] != "PAYMENT")] #filtering only transactions that are 'SALES' only
#we looked and found that returns and payments don't make up a majority of the transaction types, thus it's impact on prediction shouldn't matter that much if we solely want to predict consumer spending

transaction_data_sales['transaction_date'] = pd.to_datetime(transaction_data_sales['transaction_date'])
transaction_data_salesf = transaction_data_sales[(transaction_data_sales['transaction_date'] >= "2024-08-01") & (transaction_data_sales['transaction_date'] <= "2025-03-31")] #last 8 months

wrld_transaction_data_sales['transaction_date'] = pd.to_datetime(wrld_transaction_data_sales['transaction_date'])
wrld_transaction_data_salesf = wrld_transaction_data_sales[(wrld_transaction_data_sales['transaction_date'] >= "2024-08-01") & (wrld_transaction_data_sales['transaction_date'] <= "2025-03-31")] #last 8 months


total_sales_customer1 = transaction_data_sales.groupby('current_account_nbr')['transaction_amt'].sum().reset_index()
total_sales_customer1.rename(columns = {'transaction_amt' : 'total spent'})

total_sales_customer2 = wrld_transaction_data_sales.groupby('current_account_nbr')['transaction_amt'].sum().reset_index() 
total_sales_customer2.rename(columns = {'transaction_amt' : 'total spent'})

f_sales1 = transaction_data_salesf.groupby('current_account_nbr')['transaction_amt'].sum().reset_index()
f_sales2 = wrld_transaction_data_salesf.groupby('current_account_nbr')['transaction_amt'].sum().reset_index()

all_transactions = pd.DataFrame(columns = ['current account nbr', 'total spent']) #made new dataframe grouping account numbers together and summing up their total spending acroos the transaction_data_sales and wrld_transaction_data_sales
acc_nbrs = total_sales_customer1['current_account_nbr'].to_list() + total_sales_customer2['current_account_nbr'].to_list()
acc_total_spent = total_sales_customer1['transaction_amt'].to_list() + total_sales_customer2['transaction_amt'].to_list()

acc_spent_8 = f_sales1['transaction_amt'].to_list() + f_sales2['transaction_amt'].to_list()

all_transactions['current account nbr'] = acc_nbrs
all_transactions['total spent'] = acc_total_spent

all_transactions['total spent per month'] = all_transactions['total spent'] / 12
all_transactions['total spent per quarter'] = all_transactions['total spent'] / 3


diff = len(all_transactions) - (len(f_sales1) + len(f_sales2))
acc_spent_8.extend([0] * diff)
all_transactions['totals spent last 8 months'] = acc_spent_8
all_transactions




C:\Users\lyons\AppData\Local\Temp\ipykernel_25232\2571703414.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wrld_transaction_data_sales['transaction_date'] = pd.to_datetime(wrld_transaction_data_sales['transaction_date'])


,current account nbr,total spent,total spent per month,total spent per quarter,totals spent last 8 months
0,00gwujJkd597VxLd,25.90,2.158333,8.633333,25.90
1,00iP5U82D8XwVQ9G,4897.19,408.099167,1632.396667,3904.76
2,00oyr3QppAzjLws4,855.60,71.300000,285.200000,342.09
3,02WRVV8yusgcZWLW,824.44,68.703333,274.813333,824.44
4,033o9yHYen3xoz6k,2627.86,218.988333,875.953333,2152.96
...,...,...,...,...,...
18702,zyZhjzJwhpOgSvmc,6212.50,517.708333,2070.833333,0.00
18703,zz3nbtZXS41NZk0h,0.00,0.000000,0.000000,0.00
18704,zzBy2qNM78aRV580,2878.28,239.856667,959.426667,0.00
18705,zzEuUBBmvGiVnabb,20413.63,1701.135833,6804.543333,0.00


In [83]:
wrld_transaction_data_sales

,current_account_nbr,transaction_type,transaction_code,transaction_date,transaction_amt,payment_type,transaction_return_cnt,transaction_sale_cnt,product_amt,product_qty,...,first_purchase_ind,adj_orgn_tran_dt,curr_markup_fee,fcr_amount,fcr_flag,fcr_rate_of_exchange,frgn_curr_code,frgn_tran_amt,us_equiv_amt,posting_date
0,8DPC9336FwGf8TPz,SALE,253,2024-04-29,47.40,NaN,0,1,NaN,NaN,...,N,2024-03-17,0.0,NaN,NaN,NaN,840,47.40,0.0,2024-04-30
1,DRTiAtZURdSfMAB0,SALE,253,2024-06-25,87.13,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,87.13,0.0,2024-06-27
2,4y1CqjRDtP3w8Nb5,SALE,253,2024-08-06,30.00,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,30.00,0.0,2024-08-07
3,3dPDITyRPlzdq3wf,SALE,253,2024-09-05,32.30,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,32.30,0.0,2024-09-05
4,q1mnsZTeAaQH71AF,SALE,253,2024-12-30,4.40,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,4.40,0.0,2024-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053849,uqlll7MEoZkj3vur,SALE,253,2024-11-12,9.57,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,9.57,0.0,2024-11-13
1053850,ZGjYc8aETjtytUYR,SALE,253,2025-03-10,16.11,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,16.11,0.0,2025-03-11
1053851,4JpGONAzyfmoxi7A,SALE,253,2025-03-20,20.37,NaN,0,1,NaN,NaN,...,N,2025-02-26,0.0,NaN,NaN,NaN,840,20.37,0.0,2025-03-21
1053852,tUUUlJJZ2ETedT7g,SALE,253,2024-11-21,180.00,NaN,0,1,NaN,NaN,...,N,2024-11-12,0.0,NaN,NaN,NaN,840,180.00,0.0,2024-11-22


In [28]:
# #part 1: Build a predictive model to forecast a customer’s spending for Q4 based on past years’ spending trends and recent last 8 months of the current year.

# #start by looking at all transactional data (transaction_data) pertaining to customers that have occured within the last 8 months. filter out the ones that aren't within the last 8 months
# #clean nan values as well
#  #time series

# #filter out inactive customers? (customer_id)

# #feature engineering -> aggragate customers with respective attributes and whatnot 

# #compare lin regression / logistic regression and grade the model 
